# Load Dependencies

In [1]:
# Base libraries
import os
import pandas as pd
import numpy as np
import regex as re
import time
from dotenv import load_dotenv

# classification libraries
from openai import OpenAI, RateLimitError, APIError, Timeout, APIConnectionError

# retry libraries
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type
from collections import deque

# GPT Configuration

## Classificator Configuration

In [ ]:
load_dotenv()

api_key = os.getenv("OPENAI4_API_KEY")
if api_key is None:
    raise ValueError("OPENAI4_API_KEY not found in environment variables")
client = OpenAI(api_key=api_key)

# Classification Function
@retry(
    retry=retry_if_exception_type((RateLimitError, APIError, Timeout, APIConnectionError)),
    wait=wait_random_exponential(min=5, max=60),
    stop=stop_after_attempt(6)
)
def openAI_classificator(context, message, model="gpt-4.1-2025-04-14"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": context
                },
                {
                    "role": "user",
                    "content": message
                }
            ],
            max_tokens=10, 
            temperature=0.0,
            top_p=0.9
        )

        return response.choices[0].message.content, response.usage.total_tokens
    
    except Exception as e:
        print(e)
        match = re.search(r"Requested (\d+)\.", str(e))
        if match:
            requested_tokens = int(match.group(1))
        else:
            requested_tokens = 0
        
        return np.nan, requested_tokens

## Token Limit Configuration

In [ ]:
# Max token per minute 
MAX_TOKENS_PER_MINUTE = 450000 # Rate limit tier 2 -> 450k tokens/minute in model gpt-4.1-2025-04-14

# save (timestamp, tokens) of each request in a deque
token_window = deque()

def clean_token_window():
    #delete the values older than 60 seconds
    current_time = time.time()
    while token_window and (current_time - token_window[0][0]) > 60:
        token_window.popleft()

def wait_if_needed(new_tokens):
    clean_token_window()
    current_tokens = sum(tokens for _, tokens in token_window)

    if current_tokens + new_tokens > MAX_TOKENS_PER_MINUTE:
        excess = (current_tokens + new_tokens) - MAX_TOKENS_PER_MINUTE
        print(f"[WAIT] amount of tokens exceded ({excess}). Waiting...")

        # Calculate the time to wait based on the excess tokens
        time_to_wait = 60 - (time.time() - token_window[0][0])
        time.sleep(time_to_wait)
        clean_token_window()

## Context and Subject Definition

In [ ]:
context = "your context goes here"

In [ ]:
subject = "your subject goes here"

# Classification

In [ ]:
fileC = "your file path with comments goes here"
dfC = pd.read_csv(fileC, dtype={"classification": str, "polarity":str})

fileV = "your file path with video classification goes here"
dfV = pd.read_csv(fileV, dtype={"classification": str, "polarity":str})

In [ ]:
def get_classification_tag(video_stance):
    try:
        video_stance = int(video_stance)
    except ValueError:
        return np.nan    
    
    if video_stance == 1:
        return "completamente en desacuerdo"
    elif video_stance == 2:
        return "en desacuerdo"
    elif video_stance == 3:
        return "ni de acuerdo ni en desacuerdo"
    elif video_stance == 4:
        return "de acuerdo"
    elif video_stance == 5:
        return "completamente de acuerdo"
    else:
        return np.nan

def get_sentiment_tag(video_sentiment):
    try:
        video_sentiment = int(video_sentiment)
    except ValueError:
        return np.nan

    if video_sentiment == 1 :
        return "negativo"
    elif video_sentiment == 2:
        return "neutral"
    elif video_sentiment == 3:
        return "positivo"
    else:
        return np.nan

In [ ]:
def comment_prompt(context,subject, video_vals):
    video_stance, video_sentiment = video_vals   
    
    return f""" 
        Contexto: {context}
    
        Instrucción: Clasifica el siguiente mensaje teniendo en cuenta el contexto dado. El video tiene un sentimiento {video_sentiment} y una postura {video_stance} respecto a la siguiente afirmacion o pregunta {subject}.
        Clasifica el siguiente mensaje en dos escalas, la primera es la de Likert y la segunda es la de polaridad o sentimiento. Para la escala Likert
        ten en cuenta las siguientes opciones: 1:'Completamente en desacuerdo', 2:'En desacuerdo', 3:'Ni de acuerdo ni en desacuerdo', 4:'De acuerdo', 5:'Completamente de acuerdo'.
        Para la escala de polaridad ten en cuenta las siguientes opciones: 1:'Negativo', 2:'Neutro', 3:'Positivo'.

        Solo responde con una de las etiquetas mencionadas (solo el número) sin ningún texto adicional. Por ejemplo "1,2" o "3,1" o "5,3", donde el primer numero sea
        la escala Likert y el segundo la escala de polaridad o sentimiento.
        """

def replied_comment_promt(context,subject, video_vals, replied_message):
    video_stance, video_sentiment = video_vals   
    
    return f""" 
        Contexto: {context}
    
        Instrucción: Clasifica el siguiente mensaje teniendo en cuenta el contexto dado. El video tiene un sentimiento {video_sentiment} y una postura {video_stance} respecto a la siguiente afirmacion o pregunta {subject}.
        El mensaje a clasificar es una respuesta a otro mensaje, el mensaje al que responde es: {replied_message}.
        Clasifica el siguiente mensaje en dos escalas, la primera es la de Likert y la segunda es la de polaridad o sentimiento. Para la escala Likert
        ten en cuenta las siguientes opciones: 1:'Completamente en desacuerdo', 2:'En desacuerdo', 3:'Ni de acuerdo ni en desacuerdo', 4:'De acuerdo', 5:'Completamente de acuerdo'.
        Para la escala de polaridad ten en cuenta las siguientes opciones: 1:'Negativo', 2:'Neutro', 3:'Positivo'.

        Solo responde con una de las etiquetas mencionadas (solo el número) sin ningún texto adicional. Por ejemplo "1,2" o "3,1" o "5,3", donde el primer numero sea
        la escala Likert y el segundo la escala de polaridad o sentimiento.
        """

In [ ]:
for i,row in dfC.iterrows():
    if pd.notnull(dfC.at[i, 'classification']):
        continue
    
    if i % 10 == 0:
        print(f"Processing row {i}/{len(dfC)}")

    if i % 100 == 0:
        print(f"saving progress {i}/{len(dfC)}")
        dfC.to_csv(fileC, index=False)

    text = row['text']
    video_vals = dfV.loc[dfV['video_id'] == row['video_id'], ['classification', 'polarity']].values[0]
    video_stance = get_classification_tag(video_vals[0])
    video_sentiment = get_sentiment_tag(video_vals[1])  

    if row['is_reply']:
        try: 
            replied_message = df.loc[df['comment_id'] == row['reply_to_comment_id'], 'text'].values[0]   
            prompt = replied_comment_promt(context,subject, (video_stance, video_sentiment), replied_message)
        except:
            print(f"Error: {row['reply_to_comment_id']} not found in df ({i})")
            prompt = comment_prompt(context,subject, (video_stance, video_sentiment))        
    else:
        prompt = comment_prompt(context,subject, (video_stance, video_sentiment))
    
    try:
        # Check if is needed to wait
        estimated_tokens = 800  # This value can be adjusted (an estimated in this case is 650, but we add 150 for safety)
        wait_if_needed(estimated_tokens)

        response = openAI_classificator(prompt, text)
        classification, polarity = response[0].split(",")
        used_tokens = response[1]

        # Guarda los resultados
        dfC.at[i, 'classification'] = classification
        dfC.at[i, 'polarity'] = polarity

        # Añade a la ventana de tokens
        token_window.append((time.time(), used_tokens))

    except Exception as e:
        print(f"Error en la fila {i}: {e}")
        dfC.at[i, 'classification'] = 'error'
        dfC.at[i, 'polarity'] = 'error'
    
    if i == len(dfC):
            # Save the dataframe at the end
            dfC.to_csv(fileC, index=False)


In [ ]:
dfC.info()